In [36]:
import json
def format_answer(answer):
    if answer == "entailment":
        return "yes"
    elif answer == "contradiction":
        return "no"
    else:
        return "unknown"
f = open("snli_1.0_test.jsonl", "r")
temp = {i : json.loads(line) for i, line in enumerate(f)}
i=1
for k, v in temp.items():
    sentence1 = v["sentence1"]
    sentence2 = v["sentence2"]
    answer = format_answer(v["gold_label"])
    if len(sentence1.split()) < 15 and len(sentence2.split()) < 15:
        g = open("select_snli/sick_test_"+str(i)+".txt", "w")
        g.write(sentence1+"\n")
        g.write(sentence2)
        g.close()
        h = open("select_snli/sick_test_"+str(i)+".answer", "w")
        h.write(answer+"\n")
        h.close()
        i+=1

In [79]:
import glob
files = glob.glob("plain/sick_*.txt")
sentence1, sentence2 = [], []
for file in files:
    f = open(file, "r")
    sentences = f.readlines()
    f.close()
    sentence1.append(sentences[0].strip())
    sentence2.append(sentences[0])
sentence = sentence1 + sentence2

In [101]:
#Calculate PMI for validity of phrases
from gensim import models
import nltk
from nltk.collocations import *
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

stopset = set(stopwords.words('english')+[","])
tokens = word_tokenize("\t".join(sentence))
tokens = [w.lower() for w in tokens if not w in stopset]
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(tokens)
finder.apply_freq_filter(5)
scores = [l[1] for l in finder.score_ngrams(bigram_measures.pmi)]
meanscores = sum(scores)/len(scores)
for i in finder.score_ngrams(bigram_measures.pmi):
    if i[1] > meanscores:
        print(i)

(('bedroom', 'slipper'), 14.46263007478904)
(('dueling', 'sticks'), 14.46263007478904)
(('dusty', 'farm'), 14.46263007478904)
(('failing', 'perform'), 14.46263007478904)
(('probably', 'pine'), 14.46263007478904)
(('reserved', 'handicapped'), 14.46263007478904)
(('sneezing', 'scaring'), 14.46263007478904)
(('space', 'reserved'), 14.46263007478904)
(('twin', 'preteen'), 14.46263007478904)
(('dangling', 'straps'), 14.047592575510194)
(('extended', 'outward'), 14.047592575510194)
(('revealing', 'extravagant'), 14.047592575510194)
(('saying', 'word'), 14.047592575510194)
(('substance', 'blurry'), 14.047592575510194)
(('tiny', 'bit'), 14.047592575510194)
(('waste', 'bin'), 14.047592575510194)
(('buttered', 'loaves'), 13.725664480622832)
(('experiencing', 'new'), 13.725664480622832)
(('fancy', 'dresses'), 13.725664480622832)
(('filling', 'pitcher'), 13.725664480622832)
(('masonry', 'pyramid'), 13.725664480622832)
(('new', 'world'), 13.725664480622832)
(('scratching', 'belly'), 13.725664480622

In [1]:
import json
import glob
import re
import os
import numpy as np
import scipy as sp
import argparse
import random
import difflib

from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error, classification_report
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing, linear_model, svm
from sklearn.feature_selection import SelectFromModel
from sklearn.externals import joblib
from sklearn.cross_validation import cross_val_score

with open('features_sick_before.pickle', 'rb') as in_f:
    load_target = np.load(in_f)
    load_source = np.load(in_f)
    load_source_phrase = np.load(in_f)

def crossvalidation(clf, X_train, y_train):
    scores = cross_val_score(clf, X_train, y_train, cv=10)
    return scores.mean(), scores.std()

def classification(X_train, y_train, X_test, y_test, results=None):
    parameters = {
        'n_estimators'      : [10, 50, 100, 200, 300, 400, 500],
        'random_state'      : [0],
        'n_jobs'            : [200],
        'max_features'      : ['auto', 'log2', 'sqrt', None],
        'criterion'         : ['gini'],
        'max_depth'         : [3, 5, 10, 20, 30, 40, 50, 100]
    }
    #random forest
    #clf = make_pipeline(
    #    preprocessing.StandardScaler(),
    #    GridSearchCV(RandomForestClassifier(), parameters))
    #clf = svm.SVC(C=1.0, gamma=0.001)
    clf = svm.SVC(kernel='linear') #SVM kernel='linear'の方が良い様子
    
    clf.fit(X_train, y_train)

    #Serialize
    #joblib.dump(clf, './'+results+'/phrase.pkl')
    #clf = joblib.load('./'+results+'/phrase.pkl')

    return clf

/Users/yanakahitomi/.pyenv/versions/ccg2lambda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/yanakahitomi/.pyenv/versions/ccg2lambda/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [8]:
half = int(len(load_target)/2)
train_sources, trial_sources = load_source[:half], load_source[half:]
train_targets, trial_targets = load_target[:half], load_target[half:]
train_phrases, trial_phrases = load_source_phrase[:half], load_source_phrase[half:]
print ('test size: {0}, training size: {1}'.format(len(trial_targets), len(train_targets)))
    
# Train the regressor
clf = classification(train_sources, train_targets, trial_sources, trial_targets)

# predict
outputs = clf.predict(trial_sources)

#負例に対して正例が少なすぎる問題
trial_targets = list(map(str, trial_targets))
outputs = list(map(str, list(outputs)))
print(classification_report(trial_targets, outputs, digits=4))

test size: 40535, training size: 40535
             precision    recall  f1-score   support

        0.0     0.7962    0.9636    0.8720     31494
        1.0     0.5264    0.1409    0.2223      9041

avg / total     0.7361    0.7801    0.7271     40535



In [7]:
#SVM
#sick
test size: 52254, training size: 52253
             precision    recall  f1-score   support

        0.0     0.8428    1.0000    0.9147     44038
        1.0     0.0000    0.0000    0.0000      8216

avg / total     0.7103    0.8428    0.7709     52254

#snli
test size: 15988, training size: 15988
             precision    recall  f1-score   support

        0.0     0.8360    1.0000    0.9107     13366
        1.0     0.0000    0.0000    0.0000      2622

avg / total     0.6989    0.8360    0.7613     15988

#sick(before)

In [9]:
#installation new tensorflow
#https://github.com/tensorflow/tensorflow
#sudo pip install --upgrade tf_nightly-1.head-py3-none-any.whl

             precision    recall  f1-score   support

        0.0     0.6401    0.9339    0.7596     32328
        1.0     0.5811    0.1486    0.2367     19939

avg / total     0.6176    0.6343    0.5601     52267



In [12]:
np.set_printoptions(threshold=np.inf)
load_source_phrase2[np.where(load_target2==0)]

array(['phrase_mountain_rope', 'phrase_mountain_wall', 'phrase_climb_make',
       'phrase_climb_rock', 'phrase_climb_of', 'phrase_climb_hold',
       'phrase_dog_down', 'phrase_dog_wear', 'phrase_dog_for',
       'phrase_muzzle_striped', 'phrase_have_down', 'phrase_have_run',
       'phrase_dog_racing', 'phrase_dog_breed', 'phrase_race_run',
       'phrase_race_down', 'phrase_muzzle_uniform', 'phrase_have_wear',
       'phrase_muzzle_track', 'phrase_dog_run', 'phrase_race_wear',
       'antonym_woman_man', 'phrase_jetskus_seadoo', 'phrase_person_woman',
       'phrase_rest_watch', 'phrase_watching_watch', 'phrase_family_watch',
       'phrase_walk_run', 'phrase_wear_on', 'phrase_walk_on',
       'phrase_walk_beach', 'phrase_hat_white', 'phrase_wear_beach',
       'phrase_wear_run', 'phrase_building_people', 'phrase_to_distance',
       'phrase_to_into', 'phrase_walk_behind', 'phrase_building_distance',
       'phrase_next_distance', 'phrase_building_between',
       'phrase_walk_peopl

In [7]:
#multi perceptron is better then SVM for phrase classification
import json
import glob
import re
import os
import numpy as np
import scipy as sp
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

# define X-fold cross validation
#fold_num = 5
#seed = 0
#kfold = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
#cvscores = []

def base_model(activation="relu", optimizer="adam", out_dim=20):
    base_model = Sequential()
    base_model.add(Dense(input_dim=14, units=1))
    base_model.add(Dense(out_dim, input_dim=14, init='uniform', activation=activation))
    base_model.add(Dense(14, init='uniform', activation=activation))
    base_model.add(Dense(1, init='uniform', activation='sigmoid'))
    base_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return base_model

with open('phrase_classifier_23fd213/features.pickle', 'rb') as in_f:
    load_target2 = np.load(in_f)
    load_source2 = np.load(in_f)
    load_source_phrase2 = np.load(in_f)
    
train_x, test_x, train_y, test_y, indices_train, indices_test = train_test_split(load_source2, load_target2, load_source_phrase2, test_size=0.5)

# Define options for parameters
activation = ["relu", "sigmoid"]
optimizer = ["adam", "adagrad"]
out_dim = [20, 30]
epochs = [1, 2]
batch_size = [1, 2]

#predictions = np.round(model.predict(test_x))
#correct = test_y[:, np.newaxis]
#extracted correct phrases
#print(indices_test[np.array(predictions == correct).flatten()])

#for train, test in kfold.split(train_x, train_y):
# create model    
model = KerasClassifier(build_fn=base_model, verbose=1)
    
# grid search
param_grid = dict(activation=activation, 
                optimizer=optimizer, 
                out_dim=out_dim, 
                epochs=epochs, 
                batch_size=batch_size)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
    
# Fit the model
grid_result = grid.fit(train_x, train_y[:, np.newaxis])
    
print (grid_result.best_score_)
print (grid_result.best_params_)
    
# Evaluate
#scores = grid.evaluate(test_x, test_y)
#print("\n")
print(grid_result.best_score_)

#    cvscores.append(scores)

print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


/Users/yanakahitomi/.pyenv/versions/3.5.0/envs/ccg2lambda/lib/python3.5/site-packages/ipykernel/__main__.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, input_dim=14, activation="relu", kernel_initializer="uniform")`
/Users/yanakahitomi/.pyenv/versions/3.5.0/envs/ccg2lambda/lib/python3.5/site-packages/ipykernel/__main__.py:25: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(14, activation="relu", kernel_initializer="uniform")`
/Users/yanakahitomi/.pyenv/versions/3.5.0/envs/ccg2lambda/lib/python3.5/site-packages/ipykernel/__main__.py:26: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`
/Users/yanakahitomi/.pyenv/versions/3.5.0/envs/ccg2lambda/lib/python3.5/site-packages/ipykernel/__main__.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(30, input_dim=14, activation="relu", kernel_initializer="uniform")`
/Users/yanakahitomi/.pyenv/versions/3.5.0/envs

0.639712626108
{'epochs': 2, 'batch_size': 2, 'activation': 'relu', 'optimizer': 'adam', 'out_dim': 20}


AttributeError: 'GridSearchCV' object has no attribute 'evaluate'

In [8]:
Y = model.predict_classes(load_source2[0:1000], batch_size=1)

 941/1000 [===========================>..] - ETA: 0s

In [33]:
indices_test[np.where(predictions == correct)]

IndexError: too many indices for array

In [1]:
from keras.models import load_model
import numpy as np

model = load_model('./phrase_classifier.mm')

Using TensorFlow backend.


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [4]:
load_source2[1]

array([ 1. ,  0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ,
        0. ,  0. ,  0. ])

In [7]:
feature = [1, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
predict = np.round(model.predict(load_source2[0:1]))

In [8]:
load_source2[0:1].shape

(1, 14)

In [11]:
np.array([feature]).shape

(1, 14)